![alt text](./img/saving-stream.png)

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

In [ ]:
sc = SparkContext()

In [ ]:
spark = SparkSession(sc)

In [ ]:
ssc = StreamingContext(sc, 5)

In [ ]:
data = ssc.textFileStream('./my_logs')

In [ ]:
lines = data.map(lambda x: x.split("|"))

In [ ]:
ssc.checkpoint('./my_ckpt/')

In [ ]:
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

In [ ]:
state_wise_data = lines.map(lambda x: (x[-4], float(x[-1])))

In [ ]:
state_wise_grouped = state_wise_data.updateStateByKey(updateFunction)

![alt text](./img/checkpoint-stream.jpg)

In [ ]:
def save_rdd(rdd):
    if not rdd.isEmpty():
        rdd.toDF(["state","count"]).write.csv("my_state_counter.csv",mode="overwrite")

In [ ]:
state_wise_grouped.foreachRDD(save_rdd)
ssc.start()
ssc.awaitTermination()